In [ ]:
import nltk
import spacy
import re

from spacy.lang.en import English
nlp = English()  # use directly

# load nltk
from nltk.tokenize import sent_tokenize
# load text from 1st chapter
with open('mimic_45k_random.txt', 'r') as f: 
    data = f.read()



In [ ]:
paragraphs = data.split("\n\n")
len(paragraphs)


In [ ]:
paragraph_sentence_list = []
for paragraph in paragraphs:
    paragraph = paragraph.replace("\n", " ")
    paragraph = paragraph.replace("--", "")
    paragraph = re.sub(r'[^a-zA-Z0-9_*.,?!åäöèÅÄÖÈÉçëË]', ' ', paragraph)
    paragraph_sentence_list.append(sent_tokenize(paragraph))

len(paragraph_sentence_list)

In [ ]:
text = ""
count = 0
for paragraph in paragraph_sentence_list:
    if " ".join(paragraph).isupper():
        with open("new.txt", "w") as fw:
            fw.write(text)
        text = ""
        count += 1
        text += "\n".join(paragraph)
        text += "\n\n"
    elif "End of the Project Gutenberg EBook" in " ".join(paragraph):
        break
    else:
        text += "\n".join(paragraph)
        text += "\n\n"

In [ ]:
import pandas as pd
import re

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

data = pd.read_csv("mimicmammo.csv", usecols=[1], error_bad_lines=False)
print(len(data))
# for row in range(len(data)):
#     entry = data.iloc[row]["text"]
#     check = entry.split()
#     print([word for word in check if word.isupper()])
f = open('preprocessed' + "mimicdemo.txt", 'w+')

i = 0
for row in range(len(data)):
    entry = data.iloc[row]["text"]
    check = entry.split()
#     entry = data.iloc[100]["text"]
#     print(type(entry))
#     print("ENTRY: ")
#     print(entry)

    text = entry
    text = text.replace("_", "")
#     print("CHANGED5")
#     print(text)

    text = re.sub('\[\*\*[^\]]*\*\*\]', '<unk>', text)
#     print("CHANGED1")
#     print(text)

    text = text.replace('\n', '').replace('\r', '')
#     print("CHANGED0")
#     print(entry)

    words = ["FINAL REPORT", "MEDICAL CONDITION", "REASON FOR THIS EXAMINATION", "Reason:", "Admitting Diagnosis:",
            "REASON FOR THIS EXAMINATION:", "PROVISIONAL FINDINGS IMPRESSION (PFI):", "INDICATION:", "FINDINGS:", "NOTE:",
            "IMPRESSION:"]
    for w in range(len(words)):
        word = words[w]
        text = text.replace(word, ". "+word, 1)

    from nltk import tokenize
#     print("TOKENIZER")
#     print(tokenize.sent_tokenize(text))

    sentences = split_into_sentences(text)
#     print("SENTENCES")
#     print(sentences)
#     print("CHANGED0.5")
    for s in range(len(sentences)):
        text = sentences[s]
        text = re.sub('[\W]+', ' ', text.lower())
#         text = re.sub(" \d+", " N ", text)
        text = re.sub("\d+", " N ", text)
        text = re.sub("v", "", text)
        text = text.replace("unk", "<unk>")
        
        allow = True
        for ind in range(len(words)):
            elim = words[ind]
            elim = re.sub('[\W]+', ' ', elim.lower())
            if text == "" or text == "N x N" or text == elim or text == elim + "N" or text == "N" or text == "N cm" or text == "N mm" or text == "N cm x N" or text.startswith("<unk> rads N") == True:
#             print("SENTENCE " + str(s) + " " + text)
                allow = False
                break
        if allow == True:
            f.write(" " + text + " \n")
            print(" " + text + " ")
            i = i + 1
print(i)
f.close()



In [ ]:
f = open('preprocessed' + "mimicdemo.txt", 'r')
lines = f.readlines()
f.close()
for l in range(len(lines)):
    print(lines[l])

In [ ]:
def get_my_string():
   """Returns a string of the text"""
   f = open("preprocessedmimicdemo.txt", 'r')
   string = str(f.read())
   f.close()
   return string

print(get_my_string())

In [ ]:
import random
import math

# Configure paths to your dataset files here
DATASET_FILE = 'preprocessedmimicdemo.txt'
FILE_TRAIN = 'ppmimictrain.txt'
FILE_VALID = 'ppmimicvalidation.txt'
FILE_TESTS = 'ppmimictest.txt'

# Set to true if you want to copy first line from main
# file into each split (like CSV header)
IS_CSV = False

# Make sure it adds to 100, no error checking below
PERCENT_TRAIN = 70
PERCENT_VALID = 15
PERCENT_TESTS = 15

data = [l for l in open(DATASET_FILE, 'r')]

train_file = open(FILE_TRAIN, 'w+')
valid_file = open(FILE_VALID, 'w+')
tests_file = open(FILE_TESTS, 'w+')

if IS_CSV:
    train_file.write(data[0])
    valid_file.write(data[0])
    tests_file.write(data[0])
    data = data[1:len(data)]

num_of_data = len(data)
num_train = int((PERCENT_TRAIN/100.0)*num_of_data)
num_valid = int((PERCENT_VALID/100.0)*num_of_data)
num_tests = int((PERCENT_TESTS/100.0)*num_of_data)

data_fractions = [num_train, num_valid, num_tests]
split_data = [[],[],[]]

rand_data_ind = 0

for split_ind, fraction in enumerate(data_fractions):
    for i in range(fraction):
        rand_data_ind = random.randint(0, len(data)-1)
        split_data[split_ind].append(data[rand_data_ind])
        data.pop(rand_data_ind)

for l in split_data[0]:
    train_file.write(l)
    
for l in split_data[1]:
    valid_file.write(l)
    
for l in split_data[2]:
    tests_file.write(l)
    
train_file.close()
valid_file.close()
tests_file.close()